In [1]:
import pandas as pd
data = pd.read_csv('movie_bd_v5.xls')

In [2]:
data.drop_duplicates(subset = 'original_title', keep = 'first', inplace = True)
display(data)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,7/13/2000,6.6,2000
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,10/27/2000,6.4,2000
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,6/30/2000,4.0,2000
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...,Diane Keaton,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,Comedy|Drama,Laurence Mark Productions|Columbia Pictures Co...,2/16/2000,5.2,2000


In [3]:
profit = pd.DataFrame(list(list(data.revenue - data.budget)), columns=['profit'])
data = pd.concat([data, profit], axis=1)

In [4]:
display(data[data.profit == data.profit.min()])

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000.0,89289910.0,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149.0,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013.0,-165710090.0


In [5]:
display(len(data[data.revenue > data.budget]))

1476

In [9]:
import time
dt = pd.DataFrame()
for i in list(data.release_date):
    old_str = i.split('/')
    if 1 <= int(old_str[0]) <= 9:
        new_str = '0' + old_str[0] + '/'
    else:
        new_str = old_str[0] + '/'
    if 1 <= int(old_str[1]) <= 9:
        new_str += '0' + old_str[1] + '/'
    else:
        new_str += old_str[1] + '/'
    new_str += old_str[2][2:]
    dt = dt.append({'release_date': time.strptime(new_str, "%m/%d/%y")[:3]}, ignore_index=True)

data = data.drop(columns='release_date')
data = pd.concat([data, dt], axis=1)
display(data)

AttributeError: 'float' object has no attribute 'split'

In [58]:
df = data[(2012 <= data.release_year) & (data.release_year <= 2014)]
display(df[df.profit == df.profit.min()].original_title)

1245    The Lone Ranger
Name: original_title, dtype: object

In [59]:
display(data.genres)

0       Action|Adventure|Science Fiction|Thriller
1       Action|Adventure|Science Fiction|Thriller
2              Adventure|Science Fiction|Thriller
3        Action|Adventure|Science Fiction|Fantasy
4                           Action|Crime|Thriller
                          ...                    
1884             Adventure|Action|Science Fiction
1885                        Horror|Family|Foreign
1886     Adventure|Animation|Action|Comedy|Family
1887                                 Comedy|Drama
1888                                     Thriller
Name: genres, Length: 1889, dtype: object

In [60]:
genres = {}
k = 0

for i in data.genres:
    old_str = i.split('|')
    for j in old_str:
        genres[j] = []

genres_df = pd.DataFrame(genres)
        
for i in data.genres:
    old_str = i.split('|')
    for j in old_str:
        genres_df.loc[k, j] = 1
    k += 1

genres_df = genres_df.fillna(0)
genres_df = pd.concat([data.original_title, genres_df], axis=1)
data = data.drop(['genres'], axis=1)
data = pd.concat([data, genres_df], axis=1)
display(data)

AttributeError: 'float' object has no attribute 'split'

In [61]:
display(genres_df.sum() == genres_df.sum().max())

TypeError: '>=' not supported between instances of 'str' and 'float'

In [62]:
display(data[data.profit > 0])

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000.0,1.513529e+09,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124.0,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015.0,1.363529e+09
1,tt1392190,150000000.0,3.784364e+08,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120.0,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015.0,2.284364e+08
2,tt2908446,110000000.0,2.952382e+08,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119.0,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015.0,1.852382e+08
3,tt2488496,200000000.0,2.068178e+09,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136.0,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015.0,1.868178e+09
4,tt2820852,190000000.0,1.506249e+09,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137.0,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015.0,1.316249e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,tt0211181,85000000.0,1.836118e+08,102 Dalmatians,Glenn Close|Ioan Gruffudd|Alice Evans|Tim McIn...,Kevin Lima,Meet Two Unlikely Heroes With A Bone To Pick.,Get ready for a howling good time as an all ne...,100.0,Comedy|Family,Walt Disney Pictures|Cruella Productions,10/7/2000,5.0,2000.0,7.036004e+06
1873,tt0160009,40000000.0,3.019910e+07,The Art of War,Wesley Snipes|Donald Sutherland|Maury Chaykin|...,Christian Duguay,Who is your foe?,When ruthless terrorists threaten to bring dow...,117.0,Action|Adventure|Thriller,Amen Ra Films|Franchise Pictures|Warner Bros.|...,8/23/2000,5.7,2000.0,4.700000e+06
1875,tt0219854,65000000.0,6.970000e+07,The Kid,Bruce Willis|Spencer Breslin|Emily Mortimer|Li...,Jon Turteltaub,Nobody ever grows up quite like they imagined.,Powerful businessman Russ Duritz is self-absor...,104.0,Fantasy|Comedy|Family,Walt Disney Pictures,7/7/2000,5.9,2000.0,7.728118e+06
1881,tt0213790,24000000.0,1.370000e+07,The Ladies Man,Tim Meadows|Karyn Parsons|Billy Dee Williams|J...,Reginald Hudlin,He's cool. He's clean. He's a love machine.,"Because of his salacious language, late-night ...",84.0,Comedy,SNL Studios,10/13/2000,5.4,2000.0,1.261000e+07


In [63]:
directors = {}
for i in data.director:
    old_str = i.split('|')
    for j in old_str:
        directors.setdefault(j, 0)

for i in data.director:
    old_str = i.split('|')
    for j in old_str:
        if len(data[(data.director == j) & (data.Action == 1)]) > 0:
            directors[j] += 1
            
for key, value in directors.items():
    if value == max(directors.values()):
        display(key)


AttributeError: 'float' object has no attribute 'split'

In [79]:
actors = {}
for i in data.cast:
    old_str = i.split('|')
    for j in old_str:
        actors.setdefault(j, 0)
        
df = data[data.release_year == 2012]
for i in list(df.index):
    for j in df.loc[i].cast.split('|'):
        actors[j] += df.loc[i].revenue

for key, value in actors.items():
    if value == max(actors.values()):
        display(key)

AttributeError: 'float' object has no attribute 'split'

In [65]:
df = data[data.budget > 50000000]

actors = {}
for i in df.cast:
    old_str = i.split('|')
    for j in old_str:
        actors.setdefault(j, 0)

for i in list(df.index):
    for j in df.loc[i].cast.split('|'):
        actors[j] += 1

for key, value in actors.items():
    if value == max(actors.values()):
        display(key)

'Matt Damon'

In [66]:
for i in genres:
    genres[i] = 0

for i in range(len(data)):
    for j in data.loc[i].cast.split('|'):
        if j == 'Nicolas Cage':
            for k in list(genres_df[genres_df.index == i].columns):
                if genres_df[genres_df.index == i][k].values[0] == 1:
                    genres[k] += 1

for key, value in genres.items():
    if value == max(genres.values()):
        display(key)

AttributeError: 'float' object has no attribute 'split'

In [67]:
df = pd.DataFrame()
for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        if j == 'Paramount Pictures':
            df = df.append(data[data.index == i], ignore_index=True)
display(df[df.profit == df.profit.min()].original_title)

AttributeError: 'float' object has no attribute 'split'

In [68]:
df = pd.DataFrame()
for i in range(len(data)):
    for j in data.loc[i].production_companies.split('|'):
        if 'Warner Bros' in j:
            df = df.append(data[data.index == i], ignore_index=True)

df1 = df.groupby(['release_year'])['revenue'].sum()
display(df1[df1 == df1.max()])

AttributeError: 'float' object has no attribute 'split'

In [69]:
months = {str(i): 0 for i in range(1, 13)}
for i in range(len(data)):
    months[str(data.release_date.loc[i][1])] += 1
display(months['6'] + months['7'] + months['8'])
        

KeyError: '/'

In [70]:
directors = {}

for i in range(len(data)):
    for j in data.loc[i].director.split('|'):
        directors[j] = 0

for i in range(len(data)):
    for j in data.loc[i].director.split('|'):
        if data.release_date.loc[i][1] == 12 or data.release_date.loc[i][1] == 1 or data.release_date.loc[i][1] == 2:
            directors[j] += 1

for key, value in directors.items():
    if value == max(directors.values()):
        display(key)

AttributeError: 'float' object has no attribute 'split'

In [22]:
df = data[['original_title', 'vote_average']].sort_values(['vote_average'], ascending=False)
df.index = list(df.original_title)
df = df['vote_average']
display(df.head(len(df)//100))

The Dark Knight                                  8.1
The Imitation Game                               8.0
Room                                             8.0
Inside Out                                       8.0
Interstellar                                     8.0
The Wolf of Wall Street                          7.9
12 Years a Slave                                 7.9
The Grand Budapest Hotel                         7.9
Memento                                          7.9
Gone Girl                                        7.9
The Pianist                                      7.9
Guardians of the Galaxy                          7.9
The Lord of the Rings: The Return of the King    7.9
Inception                                        7.9
Big Hero 6                                       7.8
The Theory of Everything                         7.8
Prisoners                                        7.8
Her                                              7.8
Name: vote_average, dtype: float64